In [82]:
import os
os.environ["GROQ_API_KEY"] = ""

In [2]:
!pip install -q langchain-groq langchain-core requests

In [3]:
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

## Step 1: Create Tool

In [4]:
@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a * b

In [5]:
print(multiply.invoke({'a':3, 'b':4}))

12


In [6]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Given 2 numbers a and b this tool returns their product
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


## Step 2: Tool Binding

In [7]:
model = ChatGroq(model="llama3-8b-8192", temperature=0.6)

In [8]:
model.invoke('hi')

AIMessage(content="Hi! It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 11, 'total_tokens': 36, 'completion_time': 0.024417996, 'prompt_time': 0.002165392, 'queue_time': 0.08291222699999999, 'total_time': 0.026583388}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_2717d04279', 'finish_reason': 'stop', 'logprobs': None}, id='run--e0a0dc16-442a-43db-99f6-38fa276c80dc-0', usage_metadata={'input_tokens': 11, 'output_tokens': 25, 'total_tokens': 36})

In [9]:
model_with_tools = model.bind_tools([multiply])

In [10]:
model_with_tools.invoke('Hi how are you')

AIMessage(content="I'm doing well, thanks for asking!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 1827, 'total_tokens': 1837, 'completion_time': 0.010604102, 'prompt_time': 0.202180434, 'queue_time': -0.266338423, 'total_time': 0.212784536}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6d1c2b0784', 'finish_reason': 'stop', 'logprobs': None}, id='run--c0456c34-acea-4dfe-9448-52b8a3bf030e-0', usage_metadata={'input_tokens': 1827, 'output_tokens': 10, 'total_tokens': 1837})

## Step 3: Tool Calling

In [11]:
query = HumanMessage('can you multiply 3 with 1000')

In [12]:
messages = [query]

In [13]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={})]

In [14]:
result = model_with_tools.invoke(messages)

In [15]:
messages.append(result)

In [16]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'a09tnvs5n', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 1874, 'total_tokens': 1947, 'completion_time': 0.071269023, 'prompt_time': 0.20761512, 'queue_time': -0.279117409, 'total_time': 0.278884143}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_2717d04279', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--9741505b-45a7-4320-b6c3-ed3a44d3ffcf-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'a09tnvs5n', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1874, 'output_tokens': 73, 'total_tokens': 1947})]

## Step 4: Tool Execution

In [17]:
tool_result = multiply.invoke(result.tool_calls[0])

In [18]:
tool_result

ToolMessage(content='3000', name='multiply', tool_call_id='a09tnvs5n')

In [19]:
messages.append(tool_result)

In [20]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'a09tnvs5n', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 1874, 'total_tokens': 1947, 'completion_time': 0.071269023, 'prompt_time': 0.20761512, 'queue_time': -0.279117409, 'total_time': 0.278884143}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_2717d04279', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--9741505b-45a7-4320-b6c3-ed3a44d3ffcf-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'a09tnvs5n', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1874, 'output_tokens': 73, 'total_tokens': 1947}),
 ToolMessage(content='3000', name='multiply', tool_call_id='a09tnvs5n')]

## Step 5: Final Output

In [21]:
model_with_tools.invoke(messages).content

'3000'

## Little Complex Example - Real time money conversion

In [22]:
from langchain_core.tools import InjectedToolArg, tool # inject tool args emphasises to add function parameter by developer instead of random
from typing import Annotated
from langchain_groq import ChatGroq
import requests

### Step 1 - Create Tool

In [23]:
@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate

In [24]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [25]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1751328001,
 'time_last_update_utc': 'Tue, 01 Jul 2025 00:00:01 +0000',
 'time_next_update_unix': 1751414401,
 'time_next_update_utc': 'Wed, 02 Jul 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 85.7501}

In [26]:
convert.invoke({'base_currency_value':10, 'conversion_rate':85.16})

851.5999999999999

### Step 2 - Tool Binding

In [71]:
llm = ChatGroq(model="llama3-8b-8192", temperature=0)

In [72]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

### Step 3 - Tool Calling

In [73]:
messages = [HumanMessage('What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd')]

In [74]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={})]

In [75]:
ai_message = llm_with_tools.invoke(messages)

In [76]:
messages.append(ai_message)

In [77]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'INR', 'target_currency': 'USD'},
  'id': 'mwkbspv6d',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10,
   'conversion_factor': {'currency': 'USD', 'value': 0.014}},
  'id': 'mm6g4149s',
  'type': 'tool_call'}]

In [78]:
ai_message

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'mwkbspv6d', 'function': {'arguments': '{"base_currency":"INR","target_currency":"USD"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'mm6g4149s', 'function': {'arguments': '{"base_currency_value":10,"conversion_factor":{"currency":"USD","value":0.014}}', 'name': 'convert'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 149, 'prompt_tokens': 1048, 'total_tokens': 1197, 'completion_time': 0.147279113, 'prompt_time': 0.116412388, 'queue_time': 0.091644205, 'total_time': 0.263691501}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6d1c2b0784', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--1101d5b6-b315-4075-b4a9-f9d48198d932-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'INR', 'target_currency': 'USD'}, 'id': 'mwkbspv6d', 'type': 'tool_call'}, {'name': 'convert', 'args': {'base_currency_value': 10, 'conversion_factor': {'cu

### Step 4 - Tool Execution

In [79]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)

In [80]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'mwkbspv6d', 'function': {'arguments': '{"base_currency":"INR","target_currency":"USD"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'mm6g4149s', 'function': {'arguments': '{"base_currency_value":10,"conversion_factor":{"currency":"USD","value":0.014}}', 'name': 'convert'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 149, 'prompt_tokens': 1048, 'total_tokens': 1197, 'completion_time': 0.147279113, 'prompt_time': 0.116412388, 'queue_time': 0.091644205, 'total_time': 0.263691501}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6d1c2b0784', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--1101d5b6-b315-4075-b4a9-f9d48198d932-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'ba

### Step 5 - Final Output

In [81]:
llm_with_tools.invoke(messages)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'brffky6bx', 'function': {'arguments': '{"base_currency_value":10,"conversion_factor":0.01166}', 'name': 'convert'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 1344, 'total_tokens': 1386, 'completion_time': 0.041422717, 'prompt_time': 0.14795717, 'queue_time': 0.092078407, 'total_time': 0.189379887}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6d1c2b0784', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--2d578da0-6888-44c4-8ce6-92c3b43dc7e6-0', tool_calls=[{'name': 'convert', 'args': {'base_currency_value': 10, 'conversion_factor': 0.01166}, 'id': 'brffky6bx', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1344, 'output_tokens': 42, 'total_tokens': 1386})

In [69]:
from langchain.agents import initialize_agent, AgentType

# Step 5: Initialize the Agent ---
agent_executor = initialize_agent(
    tools=[get_conversion_factor, convert],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # using ReAct pattern
    verbose=True  # shows internal thinking
)

/tmp/ipython-input-69-627313024.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_executor = initialize_agent(


In [70]:
# --- Step 6: Run the Agent ---
user_query = "Hi how are you?"

response = agent_executor.invoke({"input": user_query})



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Final Answer",
  "action_input": "I'm doing well, thank you! I'm here to help you with currency conversions. What can I assist you with today?"
}
```


> Finished chain.
